<a href="https://colab.research.google.com/github/LuisTrassani/analisis-futbol-argentina/blob/main/Resultados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. ETL(Extraccion, Transformación y Carga)

## 1.Extracción (Extract)

In [3]:
from google.colab import drive
import pandas as pd
import chardet

# Montamos Google Drive para acceder al archivo
drive.mount('/content/drive', force_remount=True)

ruta_archivo = '/content/drive/MyDrive/results.csv'  or '/content/results.csv'
with open(ruta_archivo, 'rb') as f:
  result = chardet.detect(f.read())
  print (result)
try:
  df = pd.read_csv(ruta_archivo, encoding=result['encoding'], sep=',', decimal='.')
  print('Extración de datos exitosa')
except FileNotFoundError:
  print(f'El archivo no se encuentra en la ruta: {ruta_archivo}')
except Exception as e:
  print(f'Ocurrió un error al leer el archivo: {e}')

Mounted at /content/drive
{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}
Extración de datos exitosa


## 2.Transformacion(Transform)

In [ ]:
 # Muestra las primeras 10 filas del DataFrame
df.head(10)

In [ ]:
# Resumen de la información del DataFrame
df.info()

In [ ]:
# Verifica tipos, en este caso es object
df.dtypes

In [ ]:
# Estadisticas descriptivas de las columnas numéricas
df.describe()

In [ ]:
# Eliminamos filas con valores faltantes críticos
df.dropna(subset=['home_team', 'away_team', 'home_score', 'away_score'], inplace=True)

# Eliminamos duplicados
df.drop_duplicates(inplace=True)
print("Transformación de datos exitosa")

In [ ]:
# Transforamar date en datatime
df['date'] = pd.to_datetime(df['date'])


print("Transformación de datos exitosa")

In [ ]:
# Renombrar columnas para facilitar el acceso y evitar caracteres especiales
df.rename(columns ={
    'date': 'Date',
    'home_team': 'Home Team',
    'away_team': 'Away Team',
    'home_score': 'Home Score',
    'away_score': 'Away Score',
    'tournament': 'Tournament',
    'city':'City',
    'country': 'Country',
    'neutral': 'Neutral'
}, inplace=True)

In [ ]:
# Diferencia de goles
df['DifGoles'] = df['Home Score'] - df['Away Score']
print(df)

In [ ]:
# Resultado del Partido (ganó, empató, perdió)
def ResultadosPartidos (row):
    if row['Home Score'] > row['Away Score']:
        return ' Ganó Local'
    elif row['Home Score'] < row['Away Score']:
        return 'Ganó Visitante'
    else:
        return 'Empate'

df['Resultado'] = df.apply(ResultadosPartidos, axis=1)


In [ ]:
df.head()

## 3.Carga (Load)

In [ ]:
# definir la ruta de salida para guardar el nuevo CSV
ruta_salida = '/content/drive/MyDrive/ResultadosFulbo.csv'

# Guardar el dataframe transformado en un nuevo archivo CSV en Drive
df.to_csv(ruta_salida, index=False, sep=',')

print(f'Archivo CSV guardado en: {ruta_salida}')

# 2. - EDA (Análisis Exploratorio de datos)


In [ ]:
from google.colab import drive
import pandas as pd
import chardet

# Montamos Google Drive para acceder al archivo
drive.mount('/content/drive', force_remount=True)

ruta_archivo = '/content/drive/MyDrive/ResultadosFulbo.csv'
with open(ruta_archivo, 'rb') as f:
  result = chardet.detect(f.read())
  print (result)
try:
  df = pd.read_csv(ruta_archivo, encoding=result['encoding'], sep=',', decimal='.')
  print('Extración de datos exitosa')
except FileNotFoundError:
  print(f'El archivo no se encuentra en la ruta: {ruta_archivo}')
except Exception as e:
  print(f'Ocurrió un error al leer el archivo: {e}')

In [ ]:
# Información general del nuevo dataset
df.info()

In [ ]:
# Estadisticas descriptivas de las columnas numéricas del nuevo DataFrame
df.describe()

In [ ]:
# ¿Podemos ver en el dataset los partidos de Argentina? Filtramos los partidos donde Argentina fue local o visitante
PartidosArgentina = df[
    (df['Home Team'] == 'Argentina') | (df['Away Team'] == 'Argentina')
]

# Mostramos las primeras filas
PartidosArgentina.head()

In [ ]:
# ¿ Cuantos partidos jugó Argentina ? Imprimimos el total de partidos Jugados de Argentina
print("Total de partidos jugados por Argentina:", len(PartidosArgentina))


In [ ]:
# ¿ Como salieron esos partidos? Determinamos el resultado desde el punto de vista de Argentina
def ResultadoArgentina(row):
    if row['Home Team'] == 'Argentina' and row['Home Score'] > row['Away Score']:
        return 'Victoria'
    elif row['Away Team'] == 'Argentina' and row['Away Score'] > row['Home Score']:
        return 'Victoria'
    elif row['Home Score'] == row['Away Score']:
        return 'Empate'
    else:
        return 'Derrota'

PartidosArgentina['ResultadoArgentina'] = PartidosArgentina.apply(ResultadoArgentina, axis=1)

# Conteo
PartidosArgentina['ResultadoArgentina'].value_counts(normalize=False) * 1

In [ ]:
# Creamos una tabla pivote donde apreciamos los partidos por fecha con total al final
tabla = pd.pivot_table(
    data=PartidosArgentina,
    index='Date',
    values=['Home Score', 'Away Score'],
    aggfunc='sum'
)

tabla.loc['Total'] = tabla.sum()
print(tabla)
